In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import learning_curve
from sklearn.metrics import accuracy_score, mean_squared_error,classification_report
import seaborn as sb

from sklearn.metrics import make_scorer, confusion_matrix
import warnings



sns.set_style('whitegrid')
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

In [18]:
survey_df = pd.read_csv('Kaggle_Salary.csv', encoding = 'UTF-8', low_memory = False)
survey_df.head()

,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,...,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT,Q10_Encoded,Q10_buckets
0,510,22-24,Male,-1,France,Master’s degree,Software Engineer,-1,"1000-9,999 employees",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,3,"30,000-39,999"
1,423,40-44,Male,-1,India,Professional degree,Software Engineer,-1,"> 10,000 employees",20+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0,"0-9,999"
2,391,40-44,Male,-1,Australia,Master’s degree,Other,0,"> 10,000 employees",20+,...,NaN,NaN,NaN,Azure SQL Database,NaN,NaN,NaN,-1,14,">250,000"
3,392,22-24,Male,-1,India,Bachelor’s degree,Other,1,0-49 employees,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0,"0-9,999"
4,470,50-54,Male,-1,France,Master’s degree,Data Scientist,-1,0-49 employees,3-4,...,NaN,AWS Relational Database Service,NaN,NaN,NaN,NaN,NaN,-1,6,"60,000-69,999"


In [19]:
col_to_keep = ["Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q11","Q14","Q15","Q16_Part_1","Q16_Part_2","Q16_Part_3","Q16_Part_4","Q16_Part_5","Q16_Part_6","Q16_Part_7","Q16_Part_8","Q16_Part_9","Q16_Part_10","Q18_Part_1","Q18_Part_2","Q18_Part_3","Q18_Part_4","Q18_Part_5","Q18_Part_6","Q18_Part_7","Q18_Part_8","Q18_Part_9","Q18_Part_10","Q19","Q22","Q21_Part_1","Q21_Part_2","Q21_Part_3","Q23","Q24_Part_1","Q24_Part_2","Q24_Part_3","Q24_Part_4","Q24_Part_5","Q24_Part_6","Q24_Part_7","Q24_Part_8","Q24_Part_9","Q24_Part_10","Q10_Encoded","Q10_buckets"]
survey_df = survey_df[col_to_keep]
survey_df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q11,Q14,...,Q24_Part_3,Q24_Part_4,Q24_Part_5,Q24_Part_6,Q24_Part_7,Q24_Part_8,Q24_Part_9,Q24_Part_10,Q10_Encoded,Q10_buckets
0,22-24,Male,France,Master’s degree,Software Engineer,"1000-9,999 employees",0,I do not know,$0 (USD),"Basic statistical software (Microsoft Excel, G...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"30,000-39,999"
1,40-44,Male,India,Professional degree,Software Engineer,"> 10,000 employees",20+,"We have well established ML methods (i.e., mod...","> $100,000 ($USD)","Cloud-based data software & APIs (AWS, GCP, Az...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"0-9,999"
2,40-44,Male,Australia,Master’s degree,Other,"> 10,000 employees",20+,I do not know,"$10,000-$99,999","Local development environments (RStudio, Jupyt...",...,NaN,NaN,NaN,NaN,Convolutional Neural Networks,NaN,NaN,NaN,14,">250,000"
3,22-24,Male,India,Bachelor’s degree,Other,0-49 employees,0,No (we do not use ML methods),$0 (USD),"Local development environments (RStudio, Jupyt...",...,"Gradient Boosting Machines (xgboost, lightgbm,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"0-9,999"
4,50-54,Male,France,Master’s degree,Data Scientist,0-49 employees,3-4,"We have well established ML methods (i.e., mod...","$10,000-$99,999","Advanced statistical software (SPSS, SAS, etc.)",...,"Gradient Boosting Machines (xgboost, lightgbm,...",Bayesian Approaches,NaN,NaN,Convolutional Neural Networks,Generative Adversarial Networks,Recurrent Neural Networks,NaN,6,"60,000-69,999"


In [20]:
survey_df['Q11'].value_counts(normalize=True)


$0 (USD)             0.329633
$100-$999            0.190612
$1000-$9,999         0.173306
$1-$99               0.121224
$10,000-$99,999      0.103510
> $100,000 ($USD)    0.081714
Name: Q11, dtype: float64

In [21]:
survey_df['Q11'].fillna(survey_df['Q11'].mode()[0],inplace=True)


In [22]:
survey_df['Q14'].value_counts(normalize=True)


Local development environments (RStudio, JupyterLab, etc.)              0.517377
Basic statistical software (Microsoft Excel, Google Sheets, etc.)       0.182850
Cloud-based data software & APIs (AWS, GCP, Azure, etc.)                0.090203
Other                                                                   0.076057
Business intelligence software (Salesforce, Tableau, Spotfire, etc.)    0.068722
Advanced statistical software (SPSS, SAS, etc.)                         0.064792
Name: Q14, dtype: float64

In [23]:
survey_df.loc[survey_df['Q14'].isnull(), 'NaN'] = 'Unknown'


In [24]:
survey_df['Q15'].value_counts(normalize=True)


3-5 years                    0.246717
1-2 years                    0.237524
< 1 years                    0.179391
5-10 years                   0.153126
10-20 years                  0.087901
I have never written code    0.048328
20+ years                    0.047015
Name: Q15, dtype: float64

In [25]:
survey_df['Q15'].fillna(survey_df['Q15'].mode()[0],inplace=True)


In [26]:
survey_df['Q19'].value_counts(normalize=True)


Python        0.769736
R             0.100723
SQL           0.069089
C++           0.012015
MATLAB        0.010795
Other         0.010607
C             0.007885
Java          0.006571
None          0.005726
Javascript    0.003473
Bash          0.002910
TypeScript    0.000469
Name: Q19, dtype: float64

In [27]:
survey_df['Q19'].fillna(survey_df['Q19'].mode()[0],inplace=True)


In [28]:
survey_df['Q22'].value_counts(normalize=True)


Never         0.806825
Once          0.092512
2-5 times     0.075355
6-24 times    0.013744
> 25 times    0.011564
Name: Q22, dtype: float64

In [29]:
survey_df['Q22'].fillna(survey_df['Q22'].mode()[0],inplace=True)


In [30]:
survey_df['Q23'].value_counts(normalize=True)


< 1 years      0.291054
1-2 years      0.258704
2-3 years      0.148847
3-4 years      0.092496
4-5 years      0.081871
5-10 years     0.079025
10-15 years    0.031306
20+ years      0.016697
Name: Q23, dtype: float64

In [31]:
survey_df['Q23'].fillna(survey_df['Q23'].mode()[0],inplace=True)


In [32]:
#Q16. Which of the following integrated development environments (IDE's) do you use on a regular basis? (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc)
integrated_development_environments = ['Q16_Part_1','Q16_Part_2','Q16_Part_3','Q16_Part_4','Q16_Part_5','Q16_Part_6','Q16_Part_7','Q16_Part_8','Q16_Part_9','Q16_Part_10']
for i in range(len(integrated_development_environments)):
    survey_df[integrated_development_environments[i]] = survey_df[integrated_development_environments[i]].apply(lambda x: 1 if (type(x) is str) else 0)
#Q18. What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python
programming_languages = ['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5','Q18_Part_6','Q18_Part_7','Q18_Part_8','Q18_Part_9','Q18_Part_10']
for i in range(len(programming_languages)):
    survey_df[programming_languages[i]] = survey_df[programming_languages[i]].apply(lambda x: 1 if (type(x) is str) else 0)
#Q21. Which types of specialized hardware do you use on a regular basis? (Select all that apply) - Selected Choice - CPUs
hardware = ['Q21_Part_1','Q21_Part_2','Q21_Part_3']
for i in range(len(hardware)):
    survey_df[hardware[i]] = survey_df[hardware[i]].apply(lambda x: 1 if (type(x) is str) else 0)
#Q24. Which of the following ML algorithms do you use on a regular basis? (Select all that apply): - Selected Choice - Linear or Logistic Regression
algorithms = ['Q24_Part_1','Q24_Part_2','Q24_Part_3','Q24_Part_4','Q24_Part_5','Q24_Part_6','Q24_Part_7','Q24_Part_8','Q24_Part_9','Q24_Part_10']
for i in range(len(algorithms)):
    survey_df[algorithms[i]] = survey_df[algorithms[i]].apply(lambda x: 1 if (type(x) is str) else 0)

In [33]:
survey_df = survey_df.rename(columns={'Q1':'H',
                       'Q2':'N',
                       'Q3':'CNOT',
                       'Q4'	:'T',
                       'Q5'	:'SWAP',
                       'Q6'	:'I',
                       'Q7'	:'T',
                       'Q8':'S',
                       'Q11':'Z',
                       'Q14':'Tdg',
                       'Q15':'Sdg',
                       'Q16_Part_1':'Rz:pi/6',
                       'Q16_Part_2':'Rz:pi/4',
                       'Q16_Part_3':'Rz:pi/2',
                       'Q16_Part_4':'Rz:pi',
                       'Q16_Part_5':'Reset',
                       'Q16_Part_6':'SX',
                       'Q16_Part_7':'SXdg',
                       'Q16_Part_8':'Y',
                       'Q16_Part_9':'Rx:pi/6',
                       'Q16_Part_10':'Rx:pi/4',
                       'Q18_Part_1' : 'Rx:pi/2',
                       'Q18_Part_2' : 'Rx:pi',
                       'Q18_Part_3' : 'Rz:pi/6',
                       'Q18_Part_4' : 'Rz:pi/4',
                       'Q18_Part_5' : 'Rz:pi/2',
                       'Q18_Part_6' : 'Rz:pi',
                       'Q18_Part_7' : 'U:pi/6',
                       'Q18_Part_8' : 'U:pi/4',
                       'Q18_Part_9' : 'U:pi/2',
                       'Q18_Part_10' : 'U:pi',
                       'Q19':'RXX',
                       'Q21_Part_1':'RZZ',
                       'Q21_Part_2':'Hardware:GPUs ',
                       'Q21_Part_3':'Hardware:TPUs ',
                       'Q22':'TPU Frequency',
                       'Q23':'ML Experience',
                       'Q24_Part_1' : 'ML algorithms: Linear or Logistic Regression',
                       'Q24_Part_2' : 'ML algorithms: Decision Trees or Random Forests',
                       'Q24_Part_3' : 'ML algorithms: Gradient Boosting Machines (xgboost, lightgbm, etc)',
                       'Q24_Part_4' : 'ML algorithms: Bayesian Approaches',
                       'Q24_Part_5' : 'ML algorithms: Evolutionary Approaches',
                       'Q24_Part_6' : 'ML algorithms: Dense Neural Networks (MLPs, etc)',
                       'Q24_Part_7' : 'ML algorithms: Convolutional Neural Networks',
                       'Q24_Part_8' : 'ML algorithms: Generative Adversarial Networks',
                       'Q24_Part_9' : 'ML algorithms: Recurrent Neural Networks',
                       'Q24_Part_10' : 'ML algorithms: Transformer Networks (BERT, gpt-2, etc)',
                       'Q10_Encoded':'Salary_Encoded',
                       'Q10_buckets':'Salary_Buckets'})

In [38]:
# Ordinal Data
survey_df['T'] = survey_df['T'].map({'I prefer not to answer':0,
                                                                 'No formal education past high school':0,
                                                                 'Some college/university study without earning a bachelor’s degree':0,
                                                                 'Professional degree':1,
                                                                 'Bachelor’s degree':2,
                                                                 'Master’s degree':3,
                                                                 'Doctoral degree':4})
                                                                
survey_df['Code Experience'] = survey_df['Code Experience'].map({'I have never written code':0,
                                                                 '< 1 years':1,
                                                                 '1-2 years':1,
                                                                 '3-5 years':1,
                                                                 '5-10 years':2,
                                                                 '10-20 years':3,
                                                                 '20+ years':4})

survey_df['ML Experience'] = survey_df['ML Experience'].map({'< 1 years':0,
                                                             '1-2 years':1,
                                                             '2-3 years':2,
                                                             '3-4 years':2,
                                                             '4-5 years':3,
                                                             '5-10 years':4,
                                                             '10-15 years':5,
                                                             '20+ years':6})

survey_df['ML Investment'] = survey_df['ML Investment'].map({'$0 (USD)':0,
                                                             '$1-$99':1,
                                                             '$100-$999':1,
                                                             '$1000-$9,999':2,
                                                             '$10,000-$99,999':3,
                                                             '> $100,000 ($USD)':4})

survey_df['Machine Learning Method'] = survey_df['Machine Learning Method'].map({'I do not know':0,
                                                                                 'No (we do not use ML methods)':0,
                                                                                 'We recently started using ML methods (i.e., models in production for less than 2 years)':1,
                                                                                 'We are exploring ML methods (and may one day put a model into production)':1,
                                                                                 'We use ML methods for generating insights (but do not put working models into production)':2,
                                                                                 'We have well established ML methods (i.e., models in production for more than 2 years)':3})

AttributeError: 'DataFrame' object has no attribute 'map'